In [ ]:
# Set up required packages
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
# Load packages
using BSeries
using SymPy
using Latexify
using LinearAlgebra

Here we study the Stormer-Verlet method applied to the Kepler Problem:

In [ ]:
# Kepler problem
function f_p(q, p)
  (-1 / sqrt(q[1]^2 + q[2]^2)^3) .* q
end

function f_q(q, p)
  p
end

Stormer-Verlet is most naturally seen as a partitioned method, but can also be cast as an additive Runge-Kutta method using the kind of splitting defined above, and with the coefficients below.

In [ ]:
# Störmer-Verlet method as additive RK method, see
# Hairer, Lubich, Wanner (2002)
# Geometric numerical integration
# Table II.2.1
As = [
    [0 0; 1//2 1//2],
    [1//2 0; 1//2 0],
]
bs = [
    [1//2, 1//2],
    [1//2, 1//2]
]
ark = AdditiveRungeKuttaMethod(As, bs)

In [ ]:
# Set up symbolic equation
dt_sym = symbols("h", real=true)
q_sym = symbols("q_1, q_2", real=true)
p_sym = symbols("p_1, p_2", real=true)
u_sym = [q_sym..., p_sym...]
f_q_sym = [f_q(q_sym, p_sym)..., 0, 0]
f_p_sym = [0, 0, f_p(q_sym, p_sym)...]
f_sym = (f_q_sym, f_p_sym)

# Compute B-series of the numerical integrator and the modified equation
truncation_order = 8
series_integrator = bseries(ark, truncation_order)
series = modified_equation(f_sym, u_sym, dt_sym, series_integrator);

#println("Modifying integrator equation for ", u_sym[1], ":")
#println(latexify(series[1], cdot=false))

We've computed the modified equation up to order $h^8$.  Now we check that it contains no terms proportional to odd powers of $h$:

In [ ]:
expr = SymPy.collect(series[1],dt_sym)
for j in 0:3
    println(expr.coeff(dt_sym,2*j+1), " ", dt_sym^(2*j+1))
end

We've only checked the flow for $q_1$; the other elements of the RHS can be checked similarly.

Next we'll compute the modifying integrator, this time only up to order $h^4$ to avoid printing out extremely long expressions:

In [ ]:
# Set up symbolic equation
dt_sym = symbols("h", real=true)
q_sym = symbols("q_1, q_2", real=true)
p_sym = symbols("p_1, p_2", real=true)
u_sym = [q_sym..., p_sym...]
f_q_sym = [f_q(q_sym, p_sym)..., 0, 0]
f_p_sym = [0, 0, f_p(q_sym, p_sym)...]
f_sym = (f_q_sym, f_p_sym)

# Compute B-series of the numerical integrator and the modified equation
truncation_order = 4
series_integrator = bseries(ark, truncation_order)
series = modifying_integrator(f_sym, u_sym, dt_sym, series_integrator);

println("Modifying integrator RHS for ", u_sym[1], ":")
series[1]

In [ ]:
println("Modifying integrator equation for ", u_sym[2], ":")
series[2]

In [ ]:
println("Modifying integrator equation for ", u_sym[3], ":")
series[3]

In [ ]:
println("Modifying integrator equation for ", u_sym[4], ":")
series[4]

Unfortunately, this system does not have the structure that enables the Stormer-Verlet method to be implemented in explicit fashion.